## 해야할 일
* 파레토 보여주는 plot
* full training 성능 비교해주는 plot

In [1]:
import plotly.graph_objects as go

plotly.offline.init_notebook_mode(connected=True)

ImportError: No module named 'plotly'

In [ ]:
# Modularize
import json
import os
import numpy as np
import plotly
import plotly.graph_objects as go

plotly.offline.init_notebook_mode(connected=True)

def json2graph(path,gr_type):
    # path : log file path
    # gr_type 0 : acc , 1: flops
    
    if gr_type == 0 : 
        gr_name ="GA acc"
    else : gr_name = "GA flops"
    
    with open(os.path.join(data_path,"train_logging.log")) as json_file:
        data = json.load(json_file)
        
    train_log = data['train_log']
    niter = len(train_log)
    npop = len(train_log['0'])
    objs =[]

    for i in range(niter):
        temp = [train_log[str(i)][j][1] for j in range(npop)]
        objs.append(temp)

    objs = np.array(objs)
    epoch = list(range(niter))

    # 0 : acc , 1: Flops
    idx = gr_type
    if gr_type == 0 : objs = -1*objs
        
    y1_lower = np.min(objs[:,:,idx],axis=-1)
    y1_lower = list(y1_lower[::-1])

    y1 = np.mean(objs[:,:,idx],axis=-1)

    y1_upper = np.max(objs[:,:,idx],axis=-1)
    y1_upper = list(y1_upper[::-1])


    x = [i+1 for i in range(niter)]
    x_rev = x[::-1]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x+x_rev,
        y=y1_upper+ y1_lower,
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name=gr_name,
    ))

    fig.add_trace(go.Scatter(
        x=x, y=y1,
        line_color='rgb(0,100,80)',
        name=gr_name,
    ))

    fig.update_traces(mode='lines')
    fig.show()

In [7]:
data_path = 'C:/Users/82103/Desktop/datascience/1219_code/basic_model/logs/__New_main_experiment_1_GA_30gen/'
json2graph(data_path,0)

## Pareto Frontier Test

In [188]:
import numpy as np
import os
import json
import sys
sys.path.insert(0,'C:/Users/82103/Desktop/datascience/1219_code/basic_model/')
from utils_kyy.create_toolbox import evaluate_v2


# Faster than is_pareto_efficient_simple, but less readable.
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = 1
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points)
        is_efficient_mask[is_efficient] = 1
        return is_efficient_mask
    else:
        return is_efficient
    
# https://stackoverflow.com/questions/32791911/fast-calculation-of-pareto-front-in-python

* log data 불러오기 -> 모델 번호/ acc / flops / Generation / 파레토 유무
* pareto 뽑기
* pareto 만 색 따로주기
* scatter 그리기

In [189]:
import pandas as pd

In [190]:
data_path = 'C:/Users/82103/Desktop/datascience/1219_code/basic_model/logs/__New_main_experiment_1_GA_30gen/'

with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]
gen_num = []
for i in range(niter):
    gen_num.extend([i for j in range(npop)])
    temp = [train_log[str(i)][j][1] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))

# 0 : acc , 1: Flops
gr_type = 0

objs[:,:,0]= -1*objs[:,:,0]

y1 = objs[:,:,0].reshape(-1).tolist()
y2 = objs[:,:,1].reshape(-1).tolist()
idxs = [i for i in range(len(y1))]
pareto = [0 for i in range(len(y1))]

In [191]:
## Data Frame으로 만들기
1. 앞에 10 Generation 에서 파레토 만들기
2. 0 ~ 20 파레토 만들기
3. 0 ~ 30 파레토 만들기



SyntaxError: invalid syntax (<ipython-input-191-42ebf1f1732e>, line 2)

In [192]:
df = pd.DataFrame({'gen':gen_num,'name': idxs, 'acc':y1, 'flops': y2})

In [193]:
df.head()

,gen,name,acc,flops
0,0,0,83.75,1.654448e+09
1,0,1,81.71,1.775048e+09
2,0,2,82.22,1.480460e+09
3,0,3,78.12,1.739114e+09
4,0,4,82.56,2.024451e+09


* 데이터 index 별로 나누기
* 나눈 걸로 파레토 각각 구하기,
* 각각 구한 파레토 OR로 합쳐주기

In [209]:
idx1 = npop*niter//3 
idx2 = npop*niter//3 * 2
idx3 = npop*niter

pareto1 = is_pareto_efficient(df[:idx1][['acc','flops']].to_numpy())
pareto1 = np.append(pareto1,[0]*(idx3-idx1))

pareto2 = is_pareto_efficient(df[:idx2][['acc','flops']].to_numpy())
pareto2 = np.append(pareto2,[0]*(idx3-idx2))

pareto3 = is_pareto_efficient(df[['acc','flops']].to_numpy())


In [201]:
pareto2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# 

In [262]:
pareto_final = np.logical_or(pareto1,pareto2, dtype ='float64')
pareto_final = np.logical_or(pareto_final,pareto3)
df['pareto'] = pareto_final

In [263]:
np.where(pareto_final==1)

(array([117, 122, 140, 194, 197, 264, 265, 267, 345, 449, 493, 513],
       dtype=int64),)

In [264]:
np.where(pareto3==1)

(array([194, 197, 264, 267, 449, 493, 513], dtype=int64),)

In [265]:
np.where(pareto2==1)

(array([194, 197, 264, 265, 267, 345], dtype=int64),)

In [266]:
np.where(pareto1==1)

(array([117, 122, 140, 194, 197], dtype=int64),)

In [301]:
df.head()

,gen,name,acc,flops,pareto
0,0,0,83.75,1.654448e+09,0
1,0,1,81.71,1.775048e+09,0
2,0,2,82.22,1.480460e+09,0
3,0,3,78.12,1.739114e+09,0
4,0,4,82.56,2.024451e+09,0


In [274]:
pareto_final.astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [275]:
df['pareto'] = pareto_final.astype(int)

In [276]:
df[df['pareto']==1]

,gen,name,acc,flops,pareto
117,5,117,74.83,1.866935e+09,1
122,6,122,76.23,1.485225e+09,1
140,7,140,77.02,1.315018e+09,1
194,9,194,79.95,1.219585e+09,1
197,9,197,80.11,1.200784e+09,1
264,13,264,64.32,1.435040e+09,1
265,13,265,75.85,1.276776e+09,1
267,13,267,79.12,1.239646e+09,1
345,17,345,73.92,1.353815e+09,1
449,22,449,69.92,1.353815e+09,1


In [277]:
df[df['pareto']==1]

,gen,name,acc,flops,pareto
117,5,117,74.83,1.866935e+09,1
122,6,122,76.23,1.485225e+09,1
140,7,140,77.02,1.315018e+09,1
194,9,194,79.95,1.219585e+09,1
197,9,197,80.11,1.200784e+09,1
264,13,264,64.32,1.435040e+09,1
265,13,265,75.85,1.276776e+09,1
267,13,267,79.12,1.239646e+09,1
345,17,345,73.92,1.353815e+09,1
449,22,449,69.92,1.353815e+09,1


In [283]:
import plotly.express as px

fig = px.scatter(df[['gen','flops','acc','pareto']], x= 'flops' , y='acc', color="pareto",
                 color_continuous_scale='bluered', hover_name="gen", hover_data=["gen"])
fig.show()

In [304]:
## Modular
import pandas as pd
import plotly.express as px


data_path='../logs/__New_main_experiment_1_RandomSerach_30gen/'

def pareto_visulize(data_path):
       
    
    with open(os.path.join(data_path,"train_logging.log")) as json_file:
        data = json.load(json_file)

    train_log = data['train_log']
    niter = len(train_log)
    npop = len(train_log['0'])
    objs =[]
    gen_num = []
    for i in range(niter):
        gen_num.extend([i for j in range(npop)])
        temp = [train_log[str(i)][j][1] for j in range(npop)]
        objs.append(temp)

    objs = np.array(objs)
    epoch = list(range(niter))

    # 0 : acc , 1: Flops
    gr_type = 0

    objs[:,:,0]= -1*objs[:,:,0]

    y1 = objs[:,:,0].reshape(-1).tolist()
    y2 = objs[:,:,1].reshape(-1).tolist()
    idxs = [i for i in range(len(y1))]
    pareto = [0 for i in range(len(y1))]
    
    df = pd.DataFrame({'gen':gen_num,'name': idxs, 'acc':y1, 'flops': y2})

    idx1 = npop*niter//3 
    idx2 = npop*niter//3 * 2
    idx3 = npop*niter

    pareto1 = is_pareto_efficient(df[:idx1][['acc','flops']].to_numpy())
    pareto1 = np.append(pareto1,[0]*(idx3-idx1))

    pareto2 = is_pareto_efficient(df[:idx2][['acc','flops']].to_numpy())
    pareto2 = np.append(pareto2,[0]*(idx3-idx2))

    pareto3 = is_pareto_efficient(df[['acc','flops']].to_numpy())

    pareto_final = np.logical_or(pareto1,pareto2, dtype ='float64')
    pareto_final = np.logical_or(pareto_final,pareto3)
    df['pareto'] = pareto_final.astype(int)
    fig = px.scatter(df[['gen','flops','acc','pareto']], x= 'flops' , y='acc', color="pareto",
                 color_continuous_scale='bluered', hover_name="gen", hover_data=["gen"])
    fig.show()
    
pareto_visulize(data_path)

### Pareto Frontier 모델 파일 떨구기

In [297]:
inds = np.where(pareto3==1)

In [289]:
data_path = 'C:/Users/82103/Desktop/datascience/1219_code/basic_model/logs/__New_main_experiment_1_GA_30gen/'
with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]

for i in range(niter):
    temp = [train_log[str(i)][j][0] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))
model_info = objs.reshape(-1,21)

In [298]:
model_info[inds]

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]])

In [305]:
data_path='../logs/__New_main_experiment_1_RandomSerach_30gen/'
with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]

for i in range(niter):
    temp = [train_log[str(i)][j][0] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))
model_info = objs.reshape(-1,21)
model_info[inds]

array([[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0]])